# Simple Census Grab

### Load a few packages.

In [ ]:
import os

import geopandas as gpd
import requests
import folium

### Get poverty data from the census.
See the possible variables, here:
http://api.census.gov/data/2014/acs5/variables.html

In [ ]:
acs  = "http://api.census.gov/data/2014/acs5?"
resp = requests.get(acs + "get=NAME,B17020_001E,B17020_002E&for=tract:*&in=state:17+county:*")
json = resp.json()

pov_df = gpd.pd.DataFrame(data = json[1:], columns = ["Name", "Population", "Poverty", "state", "county", "tract"])
pov_df['rate'] = 100 * pov_df.Poverty.astype(int) / pov_df.Population.astype(int)

### Get shapefiles of census tracts.

In [ ]:
os.system("wget http://www2.census.gov/geo/tiger/GENZ2014/shp/cb_2014_17_tract_500k.zip")
os.system("unzip cb_2014_17_tract_500k.zip")
tract_df = gpd.read_file("cb_2014_17_tract_500k.shp")

### Merge them together.

In [ ]:
merged_df = tract_df.merge(pov_df, left_on  = ["STATEFP", "COUNTYFP", "TRACTCE"], 
                                   right_on = ["state", "county", "tract"])

In [ ]:
with open('census.geojson', 'w') as f:
    f.write(merged_df.to_json())

### See it.

In [ ]:
colormap = folium.LinearColormap(("orange", "white", "purple"), vmin = 0, vmax = 50, caption = "Percent Poverty")

m = folium.Map([41.7943,-87.5907], zoom_start = 13, tiles = "cartodbpositron", max_zoom = 14, min_zoom = 6, attr = "")

folium.GeoJson(merged_df,
               style_function = lambda feature: { 
                  'fillColor': colormap(feature['properties']["rate"]) if feature["properties"]["rate"] else "k",
                  "color" : "k", "weight" : 0.3, "fillOpacity" : 0.4 if feature["properties"]["rate"] else 0,
               }).add_to(m)

colormap.add_to(m)
m